In [1]:
import torch
torch.manual_seed(42)

- 函数（function）基于变量，或者输出是否为多元
    - 单变量，单输出
    - 多变量（multi-variables），单输出
        - 神经网络，loss function
    - 单变量，多输出
    - 多变量，多输出
        - Gradient：Jacobian matrix
- 其实这里也就是在复习多元函数积分学；
- 多元函数（multivariables）微分通向矩阵分析；
    - 至少需要涉及到 jacobian （matrix）的计算；

## tensor.backward

- for non-scalar outputs
    - jacobian matrix 


$\mathbf f: R^n\rightarrow R^m, \mathbf J\in R^{m\times n}$

$$
\mathbf J = \begin{bmatrix}
  \dfrac{\partial \mathbf{f}}{\partial x_1} & \cdots & \dfrac{\partial \mathbf{f}}{\partial x_n}
\end{bmatrix}
= \begin{bmatrix}
  \nabla^{\mathrm T} f_1 \\  
  \vdots \\
  \nabla^{\mathrm T} f_m   
\end{bmatrix}
= \begin{bmatrix}
    \dfrac{\partial f_1}{\partial x_1} & \cdots & \dfrac{\partial f_1}{\partial x_n}\\
    \vdots                             & \ddots & \vdots\\
    \dfrac{\partial f_m}{\partial x_1} & \cdots & \dfrac{\partial f_m}{\partial x_n}
\end{bmatrix}
$$

In [11]:
x = torch.arange(3., requires_grad=True)
y = x*x


- $y_i=x_i^2$

$$
\mathbf y=\begin{bmatrix}
y_1\\
y_2\\
y_3\\
\end{bmatrix}=\begin{bmatrix}
x_1^2\\
x_2^2\\
x_3^2
\end{bmatrix}
$$


$$
\mathbf J_y=\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \frac{\partial y_1}{\partial x_2}\\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \frac{\partial y_2}{\partial x_2}\\
\frac{\partial y_3}{\partial x_1} & \frac{\partial y_3}{\partial x_2} & \frac{\partial y_3}{\partial x_2}
\end{bmatrix}=\begin{bmatrix}
2x_1 & 0 & 0\\
0 & 2x_2 & 0\\
0 & 0 & 2x_3
\end{bmatrix}
$$

- 变量之间没有交叉耦合关系，因此 jacobian 是对角矩阵

In [4]:
# y.backward()

$$
\mathbf J^T\cdot \mathbf v
$$

In [12]:
y.backward(gradient=torch.ones_like(x), retain_graph=True)
x.grad

tensor([0., 2., 4.])

In [13]:
y.backward(gradient=torch.tensor([1., 2., 3.]), retain_graph=True)
x.grad

tensor([ 0.,  6., 16.])

### multi input tensors

In [28]:
a = torch.tensor([1., 2.], requires_grad=True)
b = torch.tensor([2., 3.], requires_grad=True)
Q = 3*a**3 - b**2

$$
\begin{split}
&Q=\begin{bmatrix}
3a_1^3-b_1^2\\
3a_2^3-b_2^2
\end{bmatrix}\\
&\mathbf J_Q = \begin{bmatrix}
\frac{\partial Q_1}{\partial a_1} & \frac{\partial Q_1}{\partial b_1} & \frac{\partial Q_1}{\partial a_2} & \frac{\partial Q_1}{\partial b_2}\\
\frac{\partial Q_2}{\partial a_1} & \frac{\partial Q_2}{\partial b_1} & \frac{\partial Q_2}{\partial a_2} & \frac{\partial Q_2}{\partial b_2}\\
\end{bmatrix}=\begin{bmatrix}
9a_1^2 & -2b_1 & 0 & 0\\
0 & 0 & 9a_2^2 & -2b_2
\end{bmatrix}\\
&\mathbf J_q^Tv=\begin{bmatrix}
9a_1^2 & 0\\
-2b_1 & 0\\
0 & 9a_2^2\\
0 & -2b_2
\end{bmatrix}\cdot \begin{bmatrix}
1\\
1
\end{bmatrix}=\begin{bmatrix}
9a_1^2\\
-2b_1\\
9a_2^2\\
-2b_2
\end{bmatrix}
\end{split}
$$

In [40]:
Q.backward(gradient=torch.ones(2, dtype=torch.float32), retain_graph=True)
print(a.grad)
print(b.grad)
a.grad.zero_()
b.grad.zero_()

tensor([ 9., 36.])
tensor([-4., -6.])


tensor([0., 0.])

### detaching computation graph

In [42]:
x = torch.arange(4., requires_grad=True)
y = x*x
u = y
z = u*x
z.sum().backward()

$$
z=u\cdot x=y\cdot x=(x\cdot x)\cdot x=x^3
$$

```
x --> y --> u --> z
 \               /
  \-------------/
```

In [44]:
x.grad

tensor([ 0.,  3., 12., 27.])

In [45]:
x = torch.arange(4., requires_grad=True)
y = x*x
u = y.detach()
z = u*x
z.sum().backward()

$$
\begin{split}
z=u\cdot x\\
\frac{\partial z}{\partial x}=u
\end{split}
$$

In [46]:
x.grad

tensor([0., 1., 4., 9.])

In [47]:
u

tensor([0., 1., 4., 9.])

## `torch.autograd.grad`

### `grad()` 参数

- 当 output 非标量时，需要指定 grad_outputs 参数；
    - `grap_outputs.shape == outputs.shape`
    - inputs, outputs, grad_outputs
        - inputs: $x$
        - outputs: $y$
        - grad
            $$\frac{dy}{dx}$$
        - grad_outputs $$\frac{dL}{dy}$$
    - with grad_outputs（VJP: **vector-jacobian** product）

        $$
        x.grad=\frac{dL}{dy}\frac{dy}{dx}
        $$
        
- `retain_graph`:  True 则保留计算图， False则释放计算图
- `create_graph`: 若要计算高阶导数，则必须选为True 

### case1

In [8]:
x = torch.rand(3, 4)
# in_place
x.requires_grad_()

tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]], requires_grad=True)

In [9]:
torch.autograd.grad(x.sum(), x)

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),)

### case2

In [15]:
x = torch.tensor([2.0, 3.0], requires_grad=True)

# Define the multi-output function: y = [x0^2, x1^2]
y = x ** 2
torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y))[0]

tensor([4., 6.])

In [17]:
torch.autograd.functional.jacobian(lambda x: x**2, x)

tensor([[4., 0.],
        [0., 6.]])

In [19]:
x 

tensor([2., 3.], requires_grad=True)

In [28]:
y = torch.stack([x[0]**2, x[0]*x[1], x[1]**2])
print(torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), retain_graph=True))
print(torch.autograd.grad(y, x, grad_outputs=torch.tensor([1., 2., 3.])))

(tensor([7., 8.]),)
(tensor([10., 22.]),)


In [22]:
# torch.autograd.functional.jacobian(lambda x: torch.tensor([x[0]**2, x[0]*x[1], x[1]**2], requires_grad=True), x)
torch.autograd.functional.jacobian(lambda x: torch.stack([x[0]**2, x[0]*x[1], x[1]**2]), x)

tensor([[4., 0.],
        [3., 2.],
        [0., 6.]])

$$
\begin{split}
&f(\mathbf x)=\begin{bmatrix}
x^2\\
xy\\
y^2
\end{bmatrix}\\
&\mathbf J_f=\begin{bmatrix}
2x & 0\\
y & x\\
0 & 2y
\end{bmatrix}=\begin{bmatrix}
4 & 0\\
3 & 2\\
0 & 6
\end{bmatrix}\\
&vjp=\begin{bmatrix}1 & 2 & 3\end{bmatrix}\begin{bmatrix}
4 & 0\\
3 & 2\\
0 & 6
\end{bmatrix}
\end{split}
$$

### `flat_grad`

In [44]:
def flat_grad(y, x, retain_graph=False, create_graph=False):
    if create_graph:
        retain_graph = True

    g = torch.autograd.grad(y, x, retain_graph=retain_graph, create_graph=create_graph)
    g = torch.cat([t.view(-1) for t in g])
    return g

In [52]:
def f(x):
    return (x**2).sum()

In [53]:
x = torch.arange(4, dtype=torch.float32, requires_grad=True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [54]:
flat_grad(f(x), x)

tensor([0., 2., 4., 6.])

In [55]:
f(x).backward()
x.grad

tensor([0., 2., 4., 6.])

## jacobian 

$\mathbf f: R^n\rightarrow R^m, \mathbf J\in R^{m\times n}$

$$
\mathbf J = \begin{bmatrix}
  \dfrac{\partial \mathbf{f}}{\partial x_1} & \cdots & \dfrac{\partial \mathbf{f}}{\partial x_n}
\end{bmatrix}
= \begin{bmatrix}
  \nabla^{\mathrm T} f_1 \\  
  \vdots \\
  \nabla^{\mathrm T} f_m   
\end{bmatrix}
= \begin{bmatrix}
    \dfrac{\partial f_1}{\partial x_1} & \cdots & \dfrac{\partial f_1}{\partial x_n}\\
    \vdots                             & \ddots & \vdots\\
    \dfrac{\partial f_m}{\partial x_1} & \cdots & \dfrac{\partial f_m}{\partial x_n}
\end{bmatrix}
$$

In [2]:
def f(x):                                                                                             
    return x * x * torch.arange(4, dtype=torch.float)         

$$
\begin{split}
\mathbf f(\mathbf x)=
\begin{bmatrix}
f_1(x_1)\\
f_2(x_2)\\
f_3(x_3)\\
f_4(x_4)
\end{bmatrix}=\begin{bmatrix}
0\\
x_2^2\\
2 x_3^2\\
3 x_4^2
\end{bmatrix}\\
\end{split}
$$


$$
\mathbf J=\begin{bmatrix}
\dfrac{\partial f_1}{\partial x_1} & \dfrac{\partial f_1}{\partial x_2} & \dfrac{\partial f_1}{\partial x_3} & \dfrac{\partial f_1}{\partial x_4}\\
\dfrac{\partial f_2}{\partial x_1} & \dfrac{\partial f_2}{\partial x_2} & \dfrac{\partial f_2}{\partial x_3} & \dfrac{\partial f_2}{\partial x_4}\\
\dfrac{\partial f_3}{\partial x_1} & \dfrac{\partial f_3}{\partial x_2} & \dfrac{\partial f_3}{\partial x_3} & \dfrac{\partial f_3}{\partial x_4}\\
\dfrac{\partial f_4}{\partial x_1} & \dfrac{\partial f_4}{\partial x_2} & \dfrac{\partial f_4}{\partial x_3} & \dfrac{\partial f_4}{\partial x_4}\\
\end{bmatrix}=\begin{bmatrix}
0 & 0 & 0 & 0\\
0 & 2x_2 & 0 & 0\\
0 & 0 & 4x_3 & 0\\
0 & 0 & 0 & 6x_4\\
\end{bmatrix}
$$

In [17]:
def jacobian(y, x, create_graph=False):                                                               
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)                                                                 
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, retain_graph=True, create_graph=create_graph)
        # print(flat_y, x, i, grad_x)
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)   

In [18]:
x = torch.ones(4, requires_grad=True)     
x

tensor([1., 1., 1., 1.], requires_grad=True)

In [40]:
f(x)

tensor([0., 1., 2., 3.], grad_fn=<MulBackward0>)

In [43]:
f(x).reshape(-1).shape

torch.Size([4])

In [22]:
J = torch.autograd.functional.jacobian(f, x)
J

tensor([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.]])

In [20]:
jacobian(f(x), x)

tensor([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.]])

## hessian

$f : R^n \to R$

$$
\mathbf H_f=(\mathbf H_f)_{i,j} = \frac{\partial^2 f}{\partial x_i \, \partial x_j}= \begin{bmatrix}
  \dfrac{\partial^2 f}{\partial x_1^2} & \dfrac{\partial^2 f}{\partial x_1\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_1\,\partial x_n} \\[2.2ex]
  \dfrac{\partial^2 f}{\partial x_2\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_2^2} & \cdots & \dfrac{\partial^2 f}{\partial x_2\,\partial x_n} \\[2.2ex]
  \vdots & \vdots & \ddots & \vdots \\[2.2ex]
  \dfrac{\partial^2 f}{\partial x_n\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_n\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_n^2}
\end{bmatrix}
$$

In [23]:
def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)                                             

In [24]:
hessian(f(x), x)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 2., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 4., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 6.]]])

In [35]:
def f_0(x):
    return (x * x * torch.arange(4, dtype=torch.float32))[0]
def f_1(x):
    return (x * x * torch.arange(4, dtype=torch.float32))[1]
def f_2(x):
    return (x * x * torch.arange(4, dtype=torch.float32))[2]
def f_3(x):
    return (x * x * torch.arange(4, dtype=torch.float32))[3]

In [37]:
print(torch.autograd.functional.hessian(f_0, x))
print(torch.autograd.functional.hessian(f_1, x))
print(torch.autograd.functional.hessian(f_2, x))
print(torch.autograd.functional.hessian(f_3, x))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 6.]])


In [39]:
from functools import partial
def f_i(x, i):
    return (x*x*torch.arange(4, dtype=torch.float))[i]

torch.autograd.functional.hessian(partial(f_i, i=3), x)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 6.]])